In [1]:
from __future__ import division
import pandas as pd
import os
import numpy as np
import itertools
import matplotlib.pyplot as plt
%matplotlib inline

### Creates design files for the reinforcement learning task

Design files are DataFrames with the following info per trial:
1. `stim_set`: Which stimulus set is presented? [0, 1, 2]
2. `correct_stim_lr`: What is the location (on the screen) of the winning stim? [0 = left, 1=right]
3. `p_win_left`: Probability of winning if left is chosen
4. `p_win_right'`: Probability of winning if right is chosen
5. `p_win_correct`: Probability of winning if correct answer is chosen
6. `p_win_incorrect`: Probability of winning if incorrect answer is chosen
7. `jitter_1`: Jitter duration for phase 1 (in s; see below)
8. `jitter_3`: Jitter duration for phase 3 (in s; see below)
9. `jitter_5`: Jitter duration for phase 5 (in s; see below)

Trial phases:
1. Fixation cross (jittered timing) `[0.5, 0.75, 1.0, 1.25, 1.5s]`?
2. Cue (0.75s)
3. Fixation cross (jittered timing) `[0.5, 0.75, 1.0, 1.25, 1.5s]`?
4. Stimulus (2s)
5. Stimulus choice (jittered timing) `[0.5, 0.75, 1.0, 1.25, 1.5s]`?
6. Feedback (0.5s)
7. ITI (jittered timing) `[0.25, 0.5, ..., 3.25s]`?

In [2]:
def get_settings(tr=2, verbose=True):
    p_win = [[.8, .2], [.7, .3], [.6, .4]]
    n_runs = 3
    n_sessions = 2
    if tr == 2:
        n_trials = 128
        jitters = [0.5, 0.75, 1, 1.25, 1.5]
        volumes_per_trial = 4
    
    n_trials_per_stimset = n_trials/len(p_win)
    trial_duration = volumes_per_trial*tr
    total_duration = trial_duration*n_trials
    total_duration_min = total_duration/60
    total_volumes = 1 + n_trials*volumes_per_trial
    
    if verbose:
        print('Settings:\n\n\
        Sessions: {n_sessions}\n\
        Runs per session: {n_runs}\n\
        Trials per run: {n_trials}\n\
        Assuming a TR of {tr} seconds\n\
        Jitter options: {jitters} seconds\n\
        Total duration: {tr}*{trial_duration}*{n_trials} = {total_duration} seconds = {total_duration_min} min\n\
        Total number of volumes necessary: 1+{n_trials}*{volumes_per_trial} = {total_volumes} + warm-up pulses'.format(**locals()))
        
    return({'jitter': jitters,
            'n_trials': n_trials})

In [52]:
def generate_block_design(n_trials, jitters, 
                          mr_design=True, 
                          include_cue=True, 
                          phase_durations=['jittered', 0.75, 'jittered', 2, 'jittered', 0.5, 'iti'],
                          trial_duration=8):
    """ Generates design for a single block.
    
    jitters: list of possible jitter durations, in seconds (e.g., [0.5, 1, 1.5])
    mr_design: bool. If False, all phases that are jittered are set to -1 and skipped in the experiment; iti is set lower
    include_cue: bool. If False, the cue phase duration will be set to -1.
    """
    stim_sets = [0, 1, 2]
    correct_stim_lr = [0, 1]
    if include_cue:
        cues = ['SPD', 'ACC']
    else:
        cues = ['']
    combs = list(itertools.product(stim_sets, correct_stim_lr, cues))
    
    # make basic df
    design = pd.DataFrame(combs * int(np.ceil((n_trials/len(combs)))), 
                          columns=['stimulus_set', 'correct_stim_lr', 'cue'])
    # randomize
    design = design.sample(frac=1).reset_index(drop=True)#.reset_index('trial_ID')
    n_trials_real = design.shape[0]
    design = design.iloc[:n_trials]
    
    if not n_trials == n_trials_real:
        print('WARNING: not totally balanced (%d not a multitude of %d)' %(n_trials, len(combs)))
    
    # Add probabilities (left/right and correct/incorrect - this is redundant, I know)
    design['p_win_left'] = None
    design['p_win_right'] = None
    design['p_win_correct'] = None
    design['p_win_incorrect'] = None
    for stim_set, p_win in zip([0, 1, 2], [.8, .7, .6]):
        p_lose = 1-p_win
        design.loc[(design.stimulus_set==stim_set) & (design.correct_stim_lr == 0), 'p_win_left'] = p_win
        design.loc[(design.stimulus_set==stim_set) & (design.correct_stim_lr == 1), 'p_win_right'] = p_win
        design.loc[(design.stimulus_set==stim_set) & (design.correct_stim_lr == 1), 'p_win_left'] = p_lose
        design.loc[(design.stimulus_set==stim_set) & (design.correct_stim_lr == 0), 'p_win_right'] = p_lose
        design.loc[(design.stimulus_set==stim_set), 'p_win_correct'] = p_win
        design.loc[(design.stimulus_set==stim_set), 'p_win_incorrect'] = 1-p_win
    
    # Add phase durations
    for phase, duration in enumerate(phase_durations):
        col_key = 'phase_' + str(phase+1)
        if duration == 'jittered':
            design[col_key] = np.random.choice(jitters, size=n_trials, replace=True)
        elif duration == 'iti':
            iti_phase_col_key = 'phase_' + str(phase+1)
        else:
            design[col_key] = duration
    
    design[iti_phase_col_key] = trial_duration - design[[col for col in design.columns if 'phase' in col]].apply(sum, axis=1)

    
    if not mr_design:
        # remove jitters
        for phase, duration in enumerate(phase_durations):
            col_key = 'phase_' + str(phase+1)
            if duration == 'jittered':
                design[col_key] = -1
            # set iti to randomly sampled from [0.5, 1]
            if duration == 'iti':
                design[col_key] = np.random.uniform(0.5, 1, design.shape[0])
        # set choice highlight phase to fixed 0.5
        design['phase_5'] = 0.5

    if not include_cue:
        # always skip cue
        design['phase_2'] = -1

    return(design)

#generate_block_design(18, [0, 1, 2])

In [57]:
tr = 2
n_subjects = 1
n_runs = 3

save_dir = '../designs'
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
    
for subject_id in np.hstack([np.arange(1,n_subjects+1), 'DEBUG']):
    designs_this_session = []

    for run in range(1,n_runs+1):
        settings = get_settings(tr=tr)
        design = generate_block_design(settings['n_trials'], settings['jitter'], mr_design=False, include_cue=False)
        design['block'] = run

        designs_this_session.append(design)

    design = pd.concat(designs_this_session)
    fn = 'sub-' + str(subject_id).zfill(2) + '_tr-' + str(tr) + '_design'
    print(fn)
    design.to_csv(save_dir + '/' + fn + '.csv', sep='\t', index_label='trial_ID')

Settings:

        Sessions: 2
        Runs per session: 3
        Trials per run: 128
        Assuming a TR of 2 seconds
        Jitter options: [0.5, 0.75, 1, 1.25, 1.5] seconds
        Total duration: 2*8*128 = 1024 seconds = 17.0666666667 min
        Total number of volumes necessary: 1+128*4 = 513 + warm-up pulses
Settings:

        Sessions: 2
        Runs per session: 3
        Trials per run: 128
        Assuming a TR of 2 seconds
        Jitter options: [0.5, 0.75, 1, 1.25, 1.5] seconds
        Total duration: 2*8*128 = 1024 seconds = 17.0666666667 min
        Total number of volumes necessary: 1+128*4 = 513 + warm-up pulses
Settings:

        Sessions: 2
        Runs per session: 3
        Trials per run: 128
        Assuming a TR of 2 seconds
        Jitter options: [0.5, 0.75, 1, 1.25, 1.5] seconds
        Total duration: 2*8*128 = 1024 seconds = 17.0666666667 min
        Total number of volumes necessary: 1+128*4 = 513 + warm-up pulses
sub-01_tr-2_design
Settings:

       

In [58]:
design

,stimulus_set,correct_stim_lr,cue,p_win_left,p_win_right,p_win_correct,p_win_incorrect,phase_1,phase_2,phase_3,phase_4,phase_5,phase_6,phase_7,block
0,0,1,,0.2,0.8,0.8,0.2,-1,-1,-1,2,0.5,0.5,0.863400,1
1,0,1,,0.2,0.8,0.8,0.2,-1,-1,-1,2,0.5,0.5,0.512522,1
2,2,0,,0.6,0.4,0.6,0.4,-1,-1,-1,2,0.5,0.5,0.712982,1
3,0,1,,0.2,0.8,0.8,0.2,-1,-1,-1,2,0.5,0.5,0.755967,1
4,0,1,,0.2,0.8,0.8,0.2,-1,-1,-1,2,0.5,0.5,0.506798,1
5,2,0,,0.6,0.4,0.6,0.4,-1,-1,-1,2,0.5,0.5,0.821657,1
6,2,0,,0.6,0.4,0.6,0.4,-1,-1,-1,2,0.5,0.5,0.605349,1
7,0,0,,0.8,0.2,0.8,0.2,-1,-1,-1,2,0.5,0.5,0.797558,1
8,0,0,,0.8,0.2,0.8,0.2,-1,-1,-1,2,0.5,0.5,0.669234,1
9,2,1,,0.4,0.6,0.6,0.4,-1,-1,-1,2,0.5,0.5,0.827802,1


### design for debugging

In [4]:
tr = 2
n_subjects = 1
n_runs = 3

save_dir = '../designs'
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
    
for subject_id in ['DEBUG']: #np.arange(1,n_subjects+1):
    designs_this_session = []

    for run in range(1,n_runs+1):
        settings = get_settings(tr=tr)
        design = generate_block_design(settings['n_trials'], settings['jitter'])
        design['block'] = run

        designs_this_session.append(design)

    design = pd.concat(designs_this_session)
    fn = 'sub-' + str(subject_id).zfill(2) + '_tr-' + str(tr) + '_design'
    print(fn)
    design.to_csv(save_dir + '/' + fn + '.csv', sep='\t', index_label='trial_ID')

Settings:

        Sessions: 2
        Runs per session: 3
        Trials per run: 128
        Assuming a TR of 2 seconds
        Jitter options: [0.5, 0.75, 1, 1.25, 1.5] seconds
        Total duration: 2*8*128 = 1024 seconds = 17.0666666667 min
        Total number of volumes necessary: 1+128*4 = 513 + warm-up pulses
Settings:

        Sessions: 2
        Runs per session: 3
        Trials per run: 128
        Assuming a TR of 2 seconds
        Jitter options: [0.5, 0.75, 1, 1.25, 1.5] seconds
        Total duration: 2*8*128 = 1024 seconds = 17.0666666667 min
        Total number of volumes necessary: 1+128*4 = 513 + warm-up pulses
Settings:

        Sessions: 2
        Runs per session: 3
        Trials per run: 128
        Assuming a TR of 2 seconds
        Jitter options: [0.5, 0.75, 1, 1.25, 1.5] seconds
        Total duration: 2*8*128 = 1024 seconds = 17.0666666667 min
        Total number of volumes necessary: 1+128*4 = 513 + warm-up pulses
sub-DEBUG_tr-2_design


### No MR timing

In [50]:
tr = 2
n_subjects = 1
n_runs = 3
MR_design = False


save_dir = '../designs'
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
    

for subject_id in np.hstack([np.arange(1,n_subjects+1), 'DEBUG']):
    designs_this_session = []

    for run in range(1,n_runs+1):
        settings = get_settings(tr=tr)
        design = generate_block_design(settings['n_trials'], settings['jitters'], mr_design=False, include_cue=False)
        design['block'] = run

        designs_this_session.append(design)

    design = pd.concat(designs_this_session)
    fn = 'sub-' + str(subject_id).zfill(2) + '_tr-' + str(tr) + '_design'
    print(fn)
    design.to_csv(save_dir + '/' + fn + '.csv', sep='\t', index_label='trial_ID')

Settings:

        Sessions: 2
        Runs per session: 3
        Trials per run: 128
        Assuming a TR of 2 seconds
        Jitter options: [0.5, 0.75, 1, 1.25, 1.5] seconds
        Total duration: 2*8*128 = 1024 seconds = 17.0666666667 min
        Total number of volumes necessary: 1+128*4 = 513 + warm-up pulses


KeyError: 'jitters'

In [13]:
design

,stimulus_set,correct_stim_lr,cue,p_win_left,p_win_right,p_win_correct,p_win_incorrect,jitter_1,jitter_3,jitter_5,block
0,2,0,SPD,0.6,0.4,0.6,0.4,-1,-1,-1,1
1,2,1,SPD,0.4,0.6,0.6,0.4,-1,-1,-1,1
2,1,0,ACC,0.7,0.3,0.7,0.3,-1,-1,-1,1
3,2,0,ACC,0.6,0.4,0.6,0.4,-1,-1,-1,1
4,1,1,ACC,0.3,0.7,0.7,0.3,-1,-1,-1,1
5,2,1,ACC,0.4,0.6,0.6,0.4,-1,-1,-1,1
6,1,0,ACC,0.7,0.3,0.7,0.3,-1,-1,-1,1
7,2,1,ACC,0.4,0.6,0.6,0.4,-1,-1,-1,1
8,1,1,SPD,0.3,0.7,0.7,0.3,-1,-1,-1,1
9,2,0,ACC,0.6,0.4,0.6,0.4,-1,-1,-1,1
